Set Up the Credentials
-----------------------
Set the up the credentials to submit the runs to `r1z1` cluster

This step is not needed in Databricks Notebook

In [2]:
# The CREDENTIALS has been preset with `conda env config vars set CREDENTIALS="xxs"`

# To generate CREDENTIALS for your local environment, you can use the following command:
# data = {
#     "workspace_url": "https://dbc-04ac0685-8857.staging.cloud.databricks.com/",
#     "token": "dapi338xx", 
#     "mosaic_token": "Y4x7xx",
# }
# data = json.dumps(data)
# credentials = base64.b64encode(data.encode('utf-8')).decode('utf-8')

import os
from ygong.mosaic import submit, _set_up_environment

credentials = os.environ.get("CREDENTIALS")
_set_up_environment(credentials)

Local Testing 
------------------
Run the trainer.fit locally

In [18]:
import sys
import importlib
sys.path.append(os.path.abspath('../src'))

# always reload for local updates to take effect
import train
importlib.reload(train)


<module 'train' from '/Users/yu.gong/workspace/mosaic/custom-train-demo/src/train.py'>

In [19]:

config = train.MyConfig(global_train_batch_size=1024)

train.main(config)

Files already downloaded and verified
Files already downloaded and verified


You are using a high value of `weight_decay=0.002` for the `DecoupledSGDW` optimizer. Are you sure you want to do this? Your model's weights will be multiplied by 0.998 on every step!
******************************
Config:
composer_commit_hash: None
composer_version: 0.20.1
node_name: unknown because NODENAME environment variable not set
num_cpus_per_node: 1
num_nodes: 1
rank_zero_seed: 3869696901

******************************


train          Epoch   0:    0%|| 0/196 [00:00<?, ?ba/s]         

Submit Run 
----------

run the local train.py



In [ ]:
from mcli import RunConfig

config = RunConfig(x)

submit("src/train.py", config, ScalingConfig(gpusNum=8, gpuType="a100_80gb", poolName="r1z1"), debug=True)